In [ ]:
import numpy as np
import math

In [9]:
def ssd_anchor_one_layer(img_shape,
                         feat_shape,
                         sizes,
                         ratios,
                         step,
                         offset=0.5,
                         dtype=np.float32):
    """Computer SSD default anchor boxes for one feature layer.
    Determine the relative position grid of the centers, and the relative
    width and height.
    Arguments:
      feat_shape: Feature shape, used for computing relative position grids;
      size: Absolute reference sizes;
      ratios: Ratios to use on these features;
      img_shape: Image shape, used for computing height, width relatively to the
        former;
      offset: Grid offset.
    Return:
      y, x, h, w: Relative x and y grids, and height and width.
    """
    # Compute the position grid: simple way.
    # y, x = np.mgrid[0:feat_shape[0], 0:feat_shape[1]]
    # y = (y.astype(dtype) + offset) / feat_shape[0]
    # x = (x.astype(dtype) + offset) / feat_shape[1]
    # Weird SSD-Caffe computation using steps values...
    y, x = np.mgrid[0:feat_shape[0], 0:feat_shape[1]]
    y = (y.astype(dtype) + offset) * step / img_shape[0]
    x = (x.astype(dtype) + offset) * step / img_shape[1]

    # Expand dims to support easy broadcasting.
    y = np.expand_dims(y, axis=-1)
    x = np.expand_dims(x, axis=-1)

    # Compute relative height and width.
    # Tries to follow the original implementation of SSD for the order.
    num_anchors = len(sizes) + len(ratios)
    h = np.zeros((num_anchors, ), dtype=dtype)
    w = np.zeros((num_anchors, ), dtype=dtype)
    # Add first anchor boxes with ratio=1.
    h[0] = sizes[0] / img_shape[0]
    w[0] = sizes[0] / img_shape[1]
    di = 1
    if len(sizes) > 1:
        h[1] = math.sqrt(sizes[0] * sizes[1]) / img_shape[0]
        w[1] = math.sqrt(sizes[0] * sizes[1]) / img_shape[1]
        di += 1
    for i, r in enumerate(ratios):
        h[i+di] = sizes[0] / img_shape[0] / math.sqrt(r)
        w[i+di] = sizes[0] / img_shape[1] * math.sqrt(r)
    return y, x, h, w

In [10]:
img_shape = (300, 300)
feat_shape = (5, 5)
sizes = (153., 207.)
ratios = [2, .5, 3, 1./3]
step = 64
offset = 0.5
dtype = np.float32

In [11]:
anchor_bboxes = ssd_anchor_one_layer(img_shape,feat_shape,sizes,ratios,step,offset,dtype)

In [14]:
 yref, xref, href, wref = anchor_bboxes

In [22]:
ymin = yref - href / 2.
xmin = xref - wref / 2.
ymax = yref + href / 2.
xmax = xref + wref / 2.

In [25]:
xmin.shape

(5, 5, 6)

In [18]:
href.shape

(6,)

In [20]:
wref.shape

(6,)

In [21]:
yref

array([[[ 0.10666667],
        [ 0.10666667],
        [ 0.10666667],
        [ 0.10666667],
        [ 0.10666667]],

       [[ 0.31999999],
        [ 0.31999999],
        [ 0.31999999],
        [ 0.31999999],
        [ 0.31999999]],

       [[ 0.53333336],
        [ 0.53333336],
        [ 0.53333336],
        [ 0.53333336],
        [ 0.53333336]],

       [[ 0.74666667],
        [ 0.74666667],
        [ 0.74666667],
        [ 0.74666667],
        [ 0.74666667]],

       [[ 0.95999998],
        [ 0.95999998],
        [ 0.95999998],
        [ 0.95999998],
        [ 0.95999998]]], dtype=float32)